In [1]:
import pandas as pd

In [3]:
car = pd.read_csv('cars_data.csv')

In [4]:
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [5]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [7]:
car['year'].unique()

array(['2007', '2006', '2018', '2014', '2015', '2012', '2013', '2016',
       '2010', '2017', '2008', '2011', '2019', '2009', '2005', '2000',
       '...', '150k', 'TOUR', '2003', 'r 15', '2004', 'Zest', '/-Rs',
       'sale', '1995', 'ara)', '2002', 'SELL', '2001', 'tion', 'odel',
       '2 bs', 'arry', 'Eon', 'o...', 'ture', 'emi', 'car', 'able', 'no.',
       'd...', 'SALE', 'digo', 'sell', 'd Ex', 'n...', 'e...', 'D...',
       ', Ac', 'go .', 'k...', 'o c4', 'zire', 'cent', 'Sumo', 'cab',
       't xe', 'EV2', 'r...', 'zest'], dtype=object)

# cleaning

In [10]:
backup = car.copy()

In [14]:
car = car[car['year'].str.isnumeric()]

In [16]:
car['year'] = car['year'].astype(int)

In [28]:
car = car[car['Price'] != 'Ask For Price']

In [36]:
car['kms_driven'] = car['kms_driven'].str.split(' ').str.get(0).str.replace(',', '')

In [38]:
car = car[car['kms_driven'].str.isnumeric()]

In [40]:
car['kms_driven'] = car['kms_driven'].astype(int)
car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 817 entries, 0 to 889
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        817 non-null    object
 1   company     817 non-null    object
 2   year        817 non-null    int32 
 3   Price       817 non-null    int32 
 4   kms_driven  817 non-null    int32 
 5   fuel_type   816 non-null    object
dtypes: int32(3), object(3)
memory usage: 35.1+ KB


C:\Users\PRADIP\AppData\Local\Temp\ipykernel_29640\765438154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car['kms_driven'] = car['kms_driven'].astype(int)


In [45]:
car = car[~car['fuel_type'].isna()]

In [81]:
car['name'] = car['name'].str.split(' ').str.slice(0, 3).str.join(' ')

In [53]:
car = car[car['Price'] < 6e6].reset_index(drop=True)

In [82]:
car.to_csv('Cleaned car.csv')

# Model

In [83]:
X = car.drop(columns='Price')
y = car['Price']

In [84]:
X

,name,company,year,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,36000,Diesel
4,Ford Figo,Ford,2012,41000,Diesel
...,...,...,...,...,...
810,Maruti Suzuki Ritz,Maruti,2011,50000,Petrol
811,Tata Indica V2,Tata,2009,30000,Diesel
812,Toyota Corolla Altis,Toyota,2009,132000,Petrol
813,Tata Zest XM,Tata,2018,27000,Diesel


In [85]:
y

0       80000
1      425000
2      325000
3      575000
4      175000
        ...  
810    270000
811    110000
812    300000
813    260000
814    390000
Name: Price, Length: 815, dtype: int32

In [86]:
from sklearn.model_selection import train_test_split

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [88]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [89]:
ohe = OneHotEncoder()
ohe.fit(X[['name', 'company', 'fuel_type']])

OneHotEncoder()

In [90]:
column_trans = make_column_transformer((OneHotEncoder(),['name', 'company', 'fuel_type']), remainder='passthrough', force_int_remainder_cols=False)

In [91]:
lr = LinearRegression()

In [92]:
pipe = make_pipeline(column_trans, lr)

In [93]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [94]:
y_pred = pipe.predict(X_test)

ValueError: Found unknown categories ['Skoda Superb 1.8', 'Tata Sumo Victa', 'Hyundai Verna 1.4', 'Hyundai Verna Transform', 'Mahindra Logan', 'Hyundai Accent GLX', 'Toyota Etios GD', 'BMW 7 Series', 'Skoda Yeti Ambition', 'Hyundai Creta', 'Chevrolet Enjoy 1.4', 'Tata Indigo LS', 'Maruti Suzuki Ciaz', 'Mahindra Quanto C4', 'Hyundai Elantra 1.8', 'Ford Fiesta SXi', 'Honda WR V', 'Hyundai Eon Magna', 'Tata Zest XM', 'Jeep Wrangler Unlimited', 'Audi A3 Cabriolet', 'Tata Aria Pleasure', 'Ford Endeavor 4x4', 'Skoda Rapid Elegance', 'Hyundai Verna VGT', 'Toyota Corolla H2', 'Nissan X Trail', 'Nissan Micra XL'] in column 0 during transform

In [74]:
r2_score(y_test,y_pred)

NameError: name 'y_pred' is not defined